In [1]:
import tensorflow as tf
from tqdm import tqdm
import keras
from keras.models import load_model, Sequential,Model
from keras.layers import Dense, Flatten, Softmax, Input, Reshape,Lambda,\
    Conv2D, MaxPool2D, Dropout,Concatenate,Embedding,BatchNormalization
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import re
import numpy as np
import os
import time
import json
from inception_v3 import  InceptionV3
from tqdm import tqdm
import math

from NIC import model,greedy_inference_model,image_dense_lstm,text_emb_lstm


Using TensorFlow backend.


In [2]:
fp3= "data/captions_train2014.json"
with open(fp3,'r') as f3:
    mscoco_data= json.load(f3)
    

In [3]:
all_img_name_vector=[]
all_captions=[]
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH_TRAIN=PATH_DIR + '/data/train2014/'
for annot in mscoco_data['annotations']:
    caption =  annot['caption']
   
    caption = caption.rstrip()
    caption = caption.lower()
    caption = [x.replace(".", "") for x in caption]
    caption = "".join(caption)
    caption = caption.rstrip()
    caption = "".join(caption)
    
    image_id = annot['image_id']
    full_coco_image_path = PATH_TRAIN + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)
#     
# train_captions, img_name_vector = shuffle(all_captions,
#                                           all_img_name_vector,
#                                           random_state=1)
num_examples = 400000
train_captions = all_captions[:num_examples]
img_name_vector = all_img_name_vector[:num_examples]


In [4]:
fp4= "sentence_dict2.json"
with open(fp4,"r") as f4 :
    sentiment_dict= json.load(f4)
    

In [5]:
sentiment_vec=[]
caption_vec=[]
error=0
for i in range(num_examples):
    cap= train_captions[i]
    if cap in sentiment_dict.keys():
        senti= sentiment_dict[cap]
        
    else:
        senti=0
        error+=1
        # print(cap)
    cap= '<start> ' + cap +' <end>'
    caption_vec.append(cap)
    sentiment_vec.append(senti)
    
print(error)


25687


In [6]:
image_name_train= img_name_vector[:num_examples]
senti_train= sentiment_vec[:num_examples]
caption_train= caption_vec[:num_examples]
print(caption_train[0])

<start> a very clean and well decorated empty bathroom <end>


In [7]:
def load_image(image_path):
    img= cv2.imread(image_path)
    img= cv2.resize(img,(299,299))
    return img



In [8]:
units= 512
embedding_size=511
BATCH_SIZE=80
vocab_size=12000
max_length=20


In [9]:
top_k = vocab_size-1
tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_train)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


train_seqs = tokenizer.texts_to_sequences(caption_train)
cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length+1,value=0)

print(train_seqs[0])
print(cap_vector_train[0])



[3, 2, 145, 494, 10, 619, 421, 276, 57, 4]
[  3   2 145 494  10 619 421 276  57   4   0   0   0   0   0   0   0   0
   0   0   0]


In [10]:
fname="new_out.txt"
i=0
l1=10000
senti_score=[]
senti_avg=[]
error=0
senti_var=[]
train_caption=[]
senti_cap_dict={}
with open(fname) as f1:
    while(i<l1):
        w1= f1.readline().strip()
        if not w1:
            break
        w2= f1.readline().strip()
        w1 = [x.replace(".", "") for x in w1]
        w1= [x.replace(",","") for x in w1]
        w1= [x.replace("'","") for x in w1]
        w1= [x.replace("?","") for x in w1]
        w1= "".join(x for x in w1)
        w1= w1.rstrip()
        w1= "".join(x for x in w1)

        if w2.islower():
            w1=w2
            w2=f1.readline().strip()
        a= 0
        if w2=="Negative":
            a= -1
        elif w2=="Positive":
            a= 1
        elif w2=="Very negative":
            a= -1
        elif w2=="Very positive":
            a= 1
        elif w2=="Neutral":
            a= 0
        else:
            a=0
            error+=1
            print(w2)
        senti_score.append(a)
        train_caption.append(w1)
        senti_cap_dict[w1]=a
        i+=1
# print(senti_score[0:500] )
print(len(train_caption),"train-cap_length")
print(len(senti_score),"train-senti_score")


" .
8861 train-cap_length
8861 train-senti_score


In [11]:
num_examples=8000

image_name_train= img_name_vector[:num_examples]
senti_train= senti_score[:num_examples]
caption_train= train_caption[:num_examples]
print(caption_train[0])


a plate of delicious food including french fries


In [12]:
fp2="data/senticap_dataset.json"

with open(fp2,'r') as f2:
    senticap_data= json.load(f2)
senticap_list=[]
senticap_list_raw=[]
l=0
for img_obj in senticap_data['images']:
    # if (l>20):
    #     break
    # l+=1
    img_name= img_obj['filename']
    for sent in img_obj['sentences']:
        sentiment= sent['sentiment']
        raw= sent['raw'].lower()
        w0= raw.rstrip()
        w0 = [x.replace(".", "") for x in w0]
        w0= [x.replace(",","") for x in w0]
        w0= [x.replace("'","") for x in w0]
        w0= [x.replace("?","") for x in w0]
        w0="".join(w0)
        senticap_list.append((img_name,sentiment,w0))
        senticap_list_raw.append((img_name,w0))
        
        
        

In [13]:
"""read from output of nlp treebank"""

fname="new_out.txt"
i=0
l1=10000
senti_score=[]
senti_avg=[]
error=0
senti_var=[]
train_caption=[]
senti_cap_dict={}
with open(fname) as f1:
    while(i<l1):
        w1= f1.readline().strip()
        if not w1:
            break
        w2= f1.readline().strip()
        w1 = [x.replace(".", "") for x in w1]
        w1= [x.replace(",","") for x in w1]
        w1= [x.replace("'","") for x in w1]
        w1= [x.replace("?","") for x in w1]
        w1= "".join(x for x in w1)
        w1= w1.rstrip()
        w1= "".join(x for x in w1)

        if w2.islower():
            w1=w2
            w2=f1.readline().strip()
        a= 0
        if w2=="Negative":
            a= -1
        elif w2=="Positive":
            a= 1
        elif w2=="Very negative":
            a= -1
        elif w2=="Very positive":
            a= 1
        elif w2=="Neutral":
            a= 0
        else:
            a=0
            error+=1
            print(w2)
        senti_score.append(a)
        train_caption.append(w1)
        senti_cap_dict[w1]=a
        i+=1
# print(senti_score[0:500] )
print(len(train_caption),"train-cap_length")
print(len(senti_score),"train-senti_score")


" .
8861 train-cap_length
8861 train-senti_score


In [14]:
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH= PATH_DIR+'/data/val2014/'
image_vec=[]
sentiment_vec=[]
caption_vec=[]

for item in senticap_list:
    img= item[0]
    if item[1]==1:
        senti=1
    elif item[1]==0:
        senti=-1
    else:
        senti=0

    caption= item[2]
    # print(caption+"#")
    if caption in senti_cap_dict.keys():
        new_senti= senti_cap_dict[caption]
    else:
        new_senti= 0
    img= PATH + img 
    caption= '<start> ' + caption + ' <end>'
    image_vec.append(img)
    sentiment_vec.append(new_senti)
    caption_vec.append(caption)
    
    

In [15]:
units= 512
embedding_size=511
BATCH_SIZE=80
vocab_size=12000
max_length=20
num_examples=8861

In [16]:
image_name_train= image_vec[:num_examples]
senti_train= sentiment_vec[:num_examples]
caption_train= caption_vec[:num_examples]


In [17]:
vocab_size=4500
top_k = vocab_size-1
tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_train)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


train_seqs = tokenizer.texts_to_sequences(caption_train)
cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length+1,value=0)

print(train_seqs[0])
print(cap_vector_train[0])



[3, 2, 117, 6, 201, 32, 764, 835, 703, 4]
[  3   2 117   6 201  32 764 835 703   4   0   0   0   0   0   0   0   0
   0   0   0]


In [18]:
print(img_name_vector[8500])
# img=load_image(image_name_train[0])
img=cv2.imread(img_name_vector[8500])  
print(train_captions[8500])
# img=cv2.imshow('image',img)
cv2.imshow('ImageWindow', img)
cv2.waitKey(0)
cv2.destroyAllWindows()



/home/yiyang/project/caption/caption/data/train2014/COCO_train2014_000000266289.jpg
two people sitting on top of a bench on a lush green hillside


In [23]:
def batch_generator(batch_size,units,vocab_size,train_num,start):
    input_imgs=[]
    # start=0
    # print(start,'start')
    for j in tqdm(range(train_num)):
        img= load_image( image_name_train[start+j])
        input_imgs.append(img)
    input_imgs= np.array(input_imgs)
    input_sentis= np.array(senti_train[start:start+train_num])
    input_sequences= np.array(cap_vector_train[start:start+train_num])
    
    # print(input_sequences.shape,"input_seq")
    target= input_sequences[:,1:]
    
    # target= np.expand_dims(target,axis=-1)
    # print(target.shape,"target")
    state_c= np.zeros(shape=(train_num,512))
    state_h= np.zeros(shape=(train_num,512))
    
    while True:
        for i in range(0,train_num,batch_size):
            img_features=input_imgs[i:i+batch_size,:]
            seqs= input_sequences[i:i+batch_size,0:max_length]
            state_c= np.zeros([batch_size,units])
            state_h= np.zeros([batch_size,units])
            sentiment= input_sentis[i:i+batch_size]
            
            
            target_seq= input_sequences[i:i+batch_size,1:]
            y= keras.utils.to_categorical(target_seq,vocab_size)
            # print(y.shape,"y-shape")
            yield ([img_features,seqs,sentiment,state_c, state_h],y)




In [24]:
start=0
train_num=8000
generator_train= batch_generator(BATCH_SIZE,units,vocab_size,train_num,start)


In [25]:
start2=8000
train_num2=480
generator_dev= batch_generator(BATCH_SIZE,units,vocab_size,train_num2,start2)


In [26]:
reg=1e-4
decay=0
lr=0.001
initial_epoch=0

NIC_model=model(vocab_size,max_length,reg)
# complete_model= keras.Model(image_model.input,NIC_model.output)
NIC_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = lr, decay=decay), metrics=['accuracy'])


W1005 22:48:15.775201 139620787283776 deprecation.py:506] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W1005 22:48:16.329832 139620787283776 deprecation.py:323] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W1005 22:48:16.363183 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [27]:
NIC_model.load_weights("models/NIC_model6.h5")
NIC_model.fit_generator(generator_train, steps_per_epoch=8000//BATCH_SIZE,
                        epochs=40,
                        verbose=2, 
                        validation_data=generator_dev,
                        validation_steps=100,
                        initial_epoch = 0)


  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 0/480 [00:00<?, ?it/s]

  0%|          | 24/8000 [00:00<00:33, 236.38it/s]

  5%|▍         | 22/480 [00:00<00:02, 215.01it/s]

Epoch 1/40


...

 - 34s - loss: 2.2930 - acc: 0.6049 - val_loss: 4.7530 - val_acc: 0.3326


Epoch 39/40


 - 34s - loss: 2.2835 - acc: 0.6069 - val_loss: 4.7506 - val_acc: 0.3354


Epoch 40/40


 - 34s - loss: 2.2717 - acc: 0.6111 - val_loss: 4.7728 - val_acc: 0.3354


In [28]:
NIC_model.save_weights("models/NIC_model6.h5")

In [29]:
NIC_inference = greedy_inference_model(vocab_size, max_length)
NIC_inference.load_weights("models/NIC_model6.h5", by_name = True, skip_mismatch=True)

In [30]:
test_imgs=[]
num_test=500
start=8000
for j in tqdm(range(start,start+num_test)):
    img= load_image( image_name_train[j])
    test_imgs.append(img)
test_imgs= np.array(test_imgs)
test_sentis= np.array(senti_train[start:start+num_test])
test_sequences= np.array(cap_vector_train[start:start+num_test])
# test_seq= np.ones(shape=(500,1))
# test_seq= 3* test_seq
# test_out=[]


  0%|          | 0/500 [00:00<?, ?it/s]

  5%|▍         | 23/500 [00:00<00:02, 228.65it/s]

 10%|▉         | 48/500 [00:00<00:01, 232.65it/s]

 15%|█▍        | 73/500 [00:00<00:01, 234.77it/s]

 20%|█▉        | 98/500 [00:00<00:01, 238.14it/s]

...

 82%|████████▏ | 409/500 [00:01<00:00, 250.90it/s]

 87%|████████▋ | 435/500 [00:01<00:00, 242.84it/s]

 93%|█████████▎| 463/500 [00:01<00:00, 252.12it/s]

 98%|█████████▊| 489/500 [00:01<00:00, 246.61it/s]

100%|██████████| 500/500 [00:02<00:00, 248.57it/s]

In [31]:
N = test_imgs.shape[0]

startseq = np.repeat([tokenizer.word_index['<start>']], N)
a0 = np.zeros([N, units])
c0 = np.zeros([N, units])

# output dims: [32, N, 7378]
y_preds = np.array(NIC_inference.predict([test_imgs, startseq,test_sentis, a0, c0], verbose = 1))

# output dims: [N, 32, 7378]
y_preds = np.transpose(y_preds, axes = [1,0,2])

sequences = np.argmax(y_preds, axis = -1)
sents = tokenizer.sequences_to_texts(sequences)

# if post_process:
    # post processing: 'endseq'
sents_pp = []
for sent in sents:
    if '<end>' in sent.split():
        words = sent.split()
        sents_pp.append(' '.join(words[:words.index('<end>')]))
    else:
        sents_pp.append(sent)
sents = sents_pp


 32/500 [>.............................] - ETA: 55s

 64/500 [==>...........................] - ETA: 26s

 96/500 [====>.........................] - ETA: 16s

128/500 [======>.......................] - ETA: 11s

160/500 [========>.....................] - ETA: 8s 

...

384/500 [======================>.......] - ETA: 1s

416/500 [=======================>......] - ETA: 0s

448/500 [=========================>....] - ETA: 0s

480/500 [===========================>..] - ETA: 0s

500/500 [==============================] - 6s 11ms/step


In [32]:
print(sents[400:])

['a group of people walk down a beautiful street', 'a man is sitting on a bench next to a beautiful tree', 'a man is sitting on a bench next to a beautiful tree', 'a man is sitting on a bench next to a beautiful tree', 'a man is riding a skateboard down the sidewalk next to a beautiful tree', 'a man is riding a skateboard down the sidewalk next to a beautiful tree', 'a man is riding a skateboard down the sidewalk next to a beautiful tree', 'a person on a ski slope in the beautiful snow', 'a person on a ski slope in the beautiful snow', 'a person on a ski slope in the beautiful snow', 'a group of people walk down a beautiful street', 'a group of people walk down a beautiful street', 'a group of people walk down a beautiful street', 'a group of people walk down a beautiful street', 'a group of people walk down a beautiful street', 'a group of people walk down a beautiful street', 'a couple of stupid people walk down a street in a city city', 'a couple of stupid people walk down a street 

In [19]:
base_model = InceptionV3(include_top=True, weights=None)
weights_path = 'data/image_net.h5'
# image_model = tf.keras.applications.InceptionV3(include_top=False,
#                                                 weights='imagenet')
base_model.load_weights(weights_path)
# print(base_model.output_shape)
for layer in base_model.layers[:312]:
    layer.trainable = False

new_input = base_model.layers[0].input
hidden_layer = base_model.get_layer('avg_pool').output

image_model = Model(new_input, hidden_layer)

    

W1005 22:46:31.953823 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W1005 22:46:31.967643 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W1005 22:46:31.969490 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W1005 22:46:31.979786 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



W1005 22:46:31.980213 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



W1005 22:46:32.558979 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



W1005 22:46:32.663625 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



W1005 22:46:32.962079 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [ ]:
cv2.destroyAllWindows()

In [20]:
img_num=14
 
print(img_name_vector[img_num])
# img=load_image(image_name_train[0])
img=cv2.imread(img_name_vector[img_num]) 
print(train_captions[img_num])
# img=cv2.imshow('image',img)
cv2.imshow('ImageWindow', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

 

/home/yiyang/project/caption/caption/data/train2014/COCO_train2014_000000134754.jpg
two people carrying surf boards on a beach


In [21]:
test_img= cv2.resize(img,(299,299))
test_imgs= [test_img]
test_imgs= np.array(test_imgs)
img_out= image_model.predict(test_imgs)
cv2.imshow('ImageWindow', img_out)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [22]:
print(img_out)

[[1.9343214  0.91180134 7.80577    ... 0.         0.         3.933108  ]]
